## Building a NN with SoftMax output layer

As inputs: The input layer takes ~600 cases, and their associated features, in which Chief Justice Roberts assigned the majority opinion.

As outputs: The output layer, which uses the SoftMax Activation function, assigns a probability to each Justice and predicts which Justice was assigned to write the majority opinion.

Note: This code is adapted from the Coursera Week 4 programming assignment, Neural Network Learning.

## Load Data
?? Size of input layer
?? read in data as a CSV

In [ ]:
warning('off'); addpath('../readonly/Assignment4/'); % **Change this bit later!**
load('ex4data1.mat');
m = size(X, 1);
                          % Setup the parameters you will use for this exercise
input_layer_size  = 400;  % ?? input size?
hidden_layer_size = 5;   % 5 hidden units
num_labels = 14;          % 14 labels, for each of the 14 justices who have ever been assigned to write a majority opinion   
                          % during the Roberts court. There are 9 Justices on the Supreme Court, but they sometimes retire.

% Randomly select 100 data points to display
sel = randperm(size(X, 1));
sel = sel(1:100);

displayData(X(sel, :));

## Scale and Mean-Normalize My Data
Look up from old HW and plug in here

In [ ]:
% GRADED FUNCTION: featureNormalize
function [X_norm, mu, sigma] = featureNormalize(X)

% You need to set these values correctly
X_norm = X;
mu = zeros(1, size(X, 2));
sigma = zeros(1, size(X, 2));

mu = mean(X);
sigma = std(X);
m = size(X, 1);
mu_matrix = ones(m, 1) * mu;
sigma_matrix = ones(m, 1) * sigma;
X_norm = (X - mu_matrix) ./ (sigma_matrix); 


end

In [ ]:
% Scale features and set them to zero mean
[X mu sigma] = featureNormalize(X)


## Split Data into Train, Cross-Validation, and Test Set
Split into ~600, 200, 200.

## Create helper function sigmoidGradient
?? Do I have the sigmoid function? Do I need it?

In [ ]:
% GRADED FUNCTION: sigmoidGradient
function g = sigmoidGradient(z)

g = zeros(size(z));
g = (sigmoid(z)).*(1-sigmoid(z));
    
end

## Create Cost Function, Feed Forward, and Backpropagation
?? Why is there no SoftMax activation funcion in the existing code?? This should be in the output layer, no?

In [ ]:
% GRADED FUNCTION: nnCostFunction
function [J grad] = nnCostFunction(nn_params, input_layer_size, ...
                                   hidden_layer_size, ...
                                   num_labels, ...
                                   X, y, lambda)

Theta1 = reshape(nn_params(1:hidden_layer_size * (input_layer_size + 1)), ...
                 hidden_layer_size, (input_layer_size + 1));

Theta2 = reshape(nn_params((1 + (hidden_layer_size * (input_layer_size + 1))):end), ...
                 num_labels, (hidden_layer_size + 1));

m = size(X, 1);                               % Setup some useful variables
J = 0;                                        % You need to return the following variables correctly 
Theta1_grad = zeros(size(Theta1));
Theta2_grad = zeros(size(Theta2));

% PART 1: FEED FORWARD PROPAGATION
eye_matrix = eye(num_labels);
y_matrix = eye_matrix(y,:);

a1 = [ones(m,1) X];
z2 = a1*Theta1';
a2 = sigmoid(z2);
a2 = [ones(m, 1) a2];
z3 = a2*Theta2';
a3 = sigmoid(z3);  % This should be the Softmax Activation function!

left_term = sum(sum(-y_matrix.*log(a3)));
right_term = sum(sum((1-y_matrix).*log(1-a3)));

reg_term_left = sum(sum(Theta1(:,2:end).*Theta1(:,2:end)));
reg_term_right = sum(sum(Theta2(:,2:end).*Theta2(:,2:end)));
J = (1/m)*(left_term - right_term) + (lambda/(2*m))*(reg_term_left + reg_term_right);

% PART 2: BACK PROPAGATION
d3 = a3 - y_matrix;
z2 = a1*Theta1';
d2 = d3*Theta2(:,2:end).*sigmoidGradient(z2);
Delta1 = d2'*a1;
Delta2 = d3'*a2;

Theta1_grad = (1/m)*Delta1;
Theta2_grad = (1/m)*Delta2;

% PART 3: WEIGHT REGULARIZATION
Theta1(:,1) = zeros(hidden_layer_size, 1);
reg_term_Theta1 = (lambda/m)*(Theta1);

Theta2(:,1) = zeros(num_labels, 1);
reg_term_Theta2 = (lambda/m)*(Theta2);

Theta1_grad = Theta1_grad + reg_term_Theta1;
Theta2_grad = Theta2_grad + reg_term_Theta2;


% ============================================================

grad = [Theta1_grad(:) ; Theta2_grad(:)];     % Unroll gradients


end

## Do Random initialization of weights
?? Do I need to change anything here?

In [ ]:
function W = randInitializeWeights(L_in, L_out)

W = zeros(L_out, 1 + L_in);                                 % Return the following variables correctly 

% Randomly initialize the weights to small values
epsilon_init = 0.12;
W = rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init;

end

In [ ]:
% Initialize our weights
initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size);
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels);

% Unroll parameters
initial_nn_params = [initial_Theta1(:) ; initial_Theta2(:)];

## Learn parameters using fmincg

In [ ]:
initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size);
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels);

% Unroll parameters
initial_nn_params = [initial_Theta1(:) ; initial_Theta2(:)];

In [ ]:
options = optimset('MaxIter', 50);
lambda = 1;                          %  You should also try different values of lambda
                                     % Create "short hand" for the cost function to be minimized
costFunction = @(p) nnCostFunction(p, ...
                                   input_layer_size, ...
                                   hidden_layer_size, ...
                                   num_labels, X, y, lambda);

% Now, costFunction is a function that takes in only one argument (the
% neural network parameters)
[nn_params, cost] = fmincg(costFunction, initial_nn_params, options);

% Obtain Theta1 and Theta2 back from nn_params
Theta1 = reshape(nn_params(1:hidden_layer_size * (input_layer_size + 1)), ...
                 hidden_layer_size, (input_layer_size + 1));

Theta2 = reshape(nn_params((1 + (hidden_layer_size * (input_layer_size + 1))):end), ...
                 num_labels, (hidden_layer_size + 1));

## Calculate Accuracy of Predictions on Training Set

In [ ]:
pred = predict(Theta1, Theta2, X);
Accuracy =  mean(double(pred == y)) * 100 

## Plot Learning Curves for train and validation sets
? Modify this so it's not for a linear reg?

In [ ]:
% FUNCTION: learningCurve
function [error_train, error_val] = ...
    learningCurve(X, y, Xval, yval, lambda)

% Number of training examples
m = size(X, 1);

% You need to return these values correctly
error_train = zeros(m, 1);
error_val   = zeros(m, 1);

for i = 1:m
    Xsubset = [ones(m, 1) X](1:i,:);
    Ysubset = y(1:i);
    [theta] = trainLinearReg(Xsubset, Ysubset, lambda); %will need to modify this function here!
    
    % and modify the rest of it so I'm using the nnCostFunction

    h = Xsubset*theta;
    err = h - Ysubset;
    error_sqr = err.^2;
    Jtrain = (1/(2*i))*sum(error_sqr);
    error_train(i,1) = Jtrain;

    h = [ones(size(Xval)(1), 1) Xval]*theta;
    err = h - yval;
    error_sqr = err.^2;
    Jval = (1/(2*(size(Xval)(1))))*sum(error_sqr);
    error_val(i,1) = Jval;

end

end

In [ ]:
% Plotting your results
warning("off", "all");                         % Turn off the warnings to avoid fmincg errors - Don't worry about it! 
lambda = 0;
[error_train, error_val] = ...
learningCurve([ones(m, 1) X], y, ...
                  [ones(size(Xval, 1), 1) Xval], yval, ...
                  lambda);
plot(1:m, error_train, 1:m, error_val);
title('Learning curve for linear regression')
legend('Train', 'Cross Validation')
xlabel('Number of training examples')
ylabel('Error')
axis([0 13 0 150])

fprintf('# Training Examples\tTrain Error\tCross Validation Error\n');
for i = 1:m
    fprintf('  \t%d\t\t%f\t%f\n', i, error_train(i), error_val(i));
end

## Select lambda using the cross-validation set


In [ ]:
% GRADED FUNCTION: validationCurve
function [lambda_vec, error_train, error_val] = ...
    validationCurve(X, y, Xval, yval)

% Selected values of lambda (you should not change this)
lambda_vec = [0 0.001 0.003 0.01 0.03 0.1 0.3 1 3 10]';

% You need to return these variables correctly.
error_train = zeros(length(lambda_vec), 1);
error_val = zeros(length(lambda_vec), 1);

% use my prior cost function nnCostFunction!

for i = 1:length(lambda_vec)
    lambda = lambda_vec(i);
    theta = trainLinearReg(X, y, lambda);
    ith_error_train = nnCostFunction(X, y, theta, 0);
    ith_error_val = nnCostFunction(Xval, yval, theta, 0);
    error_train(i,1) = ith_error_train;
    error_val(i,1) = ith_error_val;  
end

end


In [ ]:
% Testing your function for various lambda values
% to finally select the best lambda.

[lambda_vec, error_train, error_val] = ...
    validationCurve(X_poly, y, X_poly_val, yval);

close all;
plot(lambda_vec, error_train, lambda_vec, error_val);
legend('Train', 'Cross Validation');
xlabel('lambda');
ylabel('Error');

fprintf('lambda\t\tTrain Error\tValidation Error\n');
for i = 1:length(lambda_vec)
	fprintf(' %f\t%f\t%f\n', ...
            lambda_vec(i), error_train(i), error_val(i));
end


## Calculate error on the test set

In [ ]:
% FUNCTION: testError
function [lambda_vec, error_test] = ...
    testError(X, y, Xval, yval)

error_test = zeros(length(lambda_vec), 1);

% calculate the error on the test set using nnCostFunction

end

end
